In [58]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
user_info = pd.read_table('d:\\data\\user_info.txt',sep=',',header=0)
rong_tag = pd.read_table('d:\\data\\rong_tag.txt',sep=',',header=0)
relation1 = pd.read_table('d:\\data\\relation1.txt',sep=',',header=0)
relation2 = pd.read_table('d:\\data\\relation2.txt',sep=',',header=0)
consum = pd.read_table('d:\\data\\consumption_recode.txt',sep=',',header=0)
train =  pd.read_table('d:\\data\\train.txt',sep=',',header=0)
test = pd.read_table('d:\\data\\test.txt',sep=',',header=0)


In [260]:
train_x = train['user_id']
train_y = train['lable']
test_x = test['user_id']


In [184]:
#user_info = pd.read_table('/home/azureuser/pywork/data/user_info.txt',sep=',',header=0)
recent_user = user_info.fillna(-1)
recent_user=recent_user.replace('NONE','-1')
recent_user['age'] = recent_user.groupby(['user_id'])['age'].transform(max)
#recent_user['modifycount'] = recent_user.groupby(['user_id'])['user_id'].count()
#分组选出时间最近的用户
recent_user_1 = (recent_user.assign(temp=recent_user.groupby(['user_id'])['tm_encode'].rank(method='first', ascending=False)).query('temp<2').sort(['tm_encode','age'],ascending=False))
del recent_user_1['temp']

age_inx = recent_user_1[recent_user_1['age']=='-1']
recent_user_1[recent_user_1['age']=='-1']['age'] = recent_user_1['age'].mode()

#增加修改次数
tt = recent_user.groupby(['user_id'])['age'].count().to_frame('modifycount').reset_index()
recent_user_1 = pd.merge(recent_user_1,tt,on='user_id',how='left')

#增加rel2
r = relation2.groupby(['user1_id','user2_id']).count()
r = r.reset_index()
r = r.groupby(['user1_id'])['relation2_type'].sum().to_frame('rel2count').reset_index()
ttt = r.rename(columns ={'user1_id':'user_id'})
recent_user_1 = pd.merge(recent_user_1,ttt,on='user_id',how='left')

recent_user_1 = recent_user_1.fillna(0)

age_inx = recent_user_1[recent_user_1['age']=='-1']
recent_user_1[recent_user_1['age']=='-1']['age'] = recent_user_1['age'].mode()

train_x = train['user_id']
train_y = train['lable']
test_x = test['user_id']

train_input_x = train_x.to_frame('user_id')
test_input_x = test_x.to_frame('user_id')
train_input_x = pd.merge(train_input_x,recent_user_1,on='user_id',how='left')
test_input_x  = pd.merge(test_input_x,recent_user_1,on='user_id',how='left')
train = pd.merge(train,recent_user_1,on='user_id',how='left')

d:\Anaconda3\lib\site-packages\ipykernel\__main__.py:7: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
d:\Anaconda3\lib\site-packages\ipykernel\__main__.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [120]:
recent_user_1.head(5)

,user_id,age,sex,expect_quota,max_month_repay,occupation,education,marital_status,live_info,local_hk,...,flow,gross_profit,business_type,business_year,personnel_num,pay_type,product_id,tm_encode,modifycount,rel2count
0,dfb0da8ea1b62548ac7983dac685162a,29,1,5000,1500.0,2,2,2,0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,2,22548595,16,NaN
1,ae3545fdf790a30cdb2f3862adb483c1,34,1,10000,1000.0,5,4,3,0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,2,22547175,7,NaN
2,b1918abb632c1d6b931541c0b3760975,25,2,8000,2500.0,2,3,1,0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,2,22545784,14,NaN
3,be65b02b7a59845f2b17be62a02e5fe3,22,1,30000,3000.0,2,4,1,0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,2,22541663,7,152.0
4,166528292c15779efc9a3d591ee65f64,26,1,20000,2000.0,2,3,3,0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,2,22540481,21,NaN


In [182]:
#r = relation2.groupby(['user1_id','user2_id']).count()
#r = r.reset_index()
#r = r.groupby(['user1_id'])['relation2_type'].sum().to_frame('rel2count').reset_index()
#r = r.rename(columns ={'user1_id':'user_id'})
#r



In [185]:
rong_df = rong_tag.groupby(['user_id'])['rong_tag'].count().reset_index()
#rong_train = rong_df[rong_df['user_id'].isin(train_x)]
train_input_x.insert(23,'tags',0)
test_input_x.insert(23,'tags',0)
#recent_user.head(5)
train_input_x['tags'] = rong_df['rong_tag'].combine_first(train_input_x['tags'])
#tag_user = train_input_x.reset_index()
test_input_x['tags'] = rong_df['rong_tag'].combine_first(test_input_x['tags'])

In [186]:
rr = relation1.groupby(['user1_id'])['user2_id'].count().reset_index()
train_input_x.insert(24,'rels',0)
test_input_x.insert(24,'rels',0)
train_input_x['rels'] = rr['user2_id'].combine_first(train_input_x['rels'])
#tag_user = train_input_x.reset_index()
test_input_x['rels'] = rr['user2_id'].combine_first(test_input_x['rels'])

In [187]:
con = consum.groupby(['user_id'])['bill_id'].count().reset_index()
train_input_x.insert(25,'cons',0)
test_input_x.insert(25,'cons',0)
train_input_x['cons'] = con['bill_id'].combine_first(train_input_x['cons'])
#tag_user = train_input_x.reset_index()
test_input_x['cons'] = con['bill_id'].combine_first(test_input_x['cons'])

In [98]:
#recent_user['age'] = recent_user.groupby(['user_id'])['age'].transform(max)


In [188]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import Normalizer
from sklearn import cross_validation

lr = LogisticRegression()

In [190]:
array_feature = ['age','salary','education','marital_status','company_type','money_function','occupation','product_id','tm_encode','tags','rels','cons','modifycount']
train_input_x_1 = train_input_x[array_feature]
test_input_x_1 = test_input_x[array_feature]
#train_input_x_1 =train_input_x[[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26]]
#test_input_x_1 =test_input_x[[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26]]
X = StandardScaler().fit_transform(train_input_x_1)
test_X = StandardScaler().fit_transform(test_input_x_1)
lr.fit(X,train_y)
res = lr.predict_proba(test_X)
print(cross_validation.cross_val_score(lr,X,train_y,cv=5))

[ 0.58219573  0.60596154  0.57884615  0.5975      0.59723024]


In [253]:
from sklearn import cross_validation
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(n_estimators=100,max_features='log2',min_weight_fraction_leaf=0.1)
model.fit(X,train_y)
#scores = cross_validation.cross_val_score(model, X, train_y, cv=3)
expected = train_y
predicted = model.predict(X)

#print(model.base_estimator_)

print(metrics.classification_report(expected, predicted))

#res = model.predict_proba(test_X)




             precision    recall  f1-score   support

          0       0.73      0.36      0.48     13141
          1       0.57      0.86      0.69     12859

avg / total       0.65      0.61      0.58     26000



In [133]:
import numpy as np
result_p = res[:,1]
data = np.array(result_p)
df = pd.DataFrame(data)
test_x = pd.DataFrame(test_x)
df = df.rename(columns ={0:'probability'})
#dataframe命名
d = test_x.join(df)
d.to_csv('d:\\data\\test_with_random.txt',index=False,header=True)

In [252]:
from sklearn.ensemble import AdaBoostClassifier
model = AdaBoostClassifier()
model.fit(X,train_y)

expected = train_y
predicted = model.predict(X)

print(metrics.classification_report(expected, predicted))

             precision    recall  f1-score   support

          0       0.70      0.43      0.53     13141
          1       0.58      0.81      0.68     12859

avg / total       0.64      0.62      0.60     26000



In [247]:
from sklearn.naive_bayes import GaussianNB
model = GaussianNB()
model.fit(X,train_y)

expected = train_y
predicted = model.predict(X)

print(metrics.classification_report(expected, predicted))


             precision    recall  f1-score   support

          0       0.63      0.40      0.49     13141
          1       0.55      0.76      0.64     12859

avg / total       0.59      0.58      0.56     26000



In [246]:
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=3)
neigh.fit(X, train_y)

expected = train_y
predicted = model.predict(X)

print(metrics.classification_report(expected, predicted))
print(metrics.confusion_matrix(expected, predicted))

score = cross_validation.cross_val_score(model, X, train_y, cv=5)
score

             precision    recall  f1-score   support

          0       0.74      0.40      0.52     13141
          1       0.58      0.86      0.70     12859

avg / total       0.66      0.63      0.61     26000

[[ 5278  7863]
 [ 1819 11040]]


array([ 0.60796001,  0.61807692,  0.60538462,  0.60980769,  0.60703982])

In [245]:
from sklearn import metrics
from sklearn.svm import SVC
# fit a SVM model to the data
model = SVC()
model.fit(X, train_y)
print(model)
# make predictions
expected = train_y
predicted = model.predict(X)
# summarize the fit of the model
print(metrics.classification_report(expected, predicted))
print(metrics.confusion_matrix(expected, predicted))
score = cross_validation.cross_val_score(model, X, train_y, cv=5)
score

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)
             precision    recall  f1-score   support

          0       0.74      0.40      0.52     13141
          1       0.58      0.86      0.70     12859

avg / total       0.66      0.63      0.61     26000

[[ 5278  7863]
 [ 1819 11040]]


array([ 0.60796001,  0.61807692,  0.60538462,  0.60980769,  0.60703982])

In [233]:
tt = train_input_x
train_1 = train[train['lable']==1]
train_0 = train[train['lable']==0]

In [234]:
ttt_1 = tt[tt['user_id'].isin(train_1['user_id'])]
ttt_0 = tt[tt['user_id'].isin(train_0['user_id'])]

In [235]:
ttt_1[ttt_1['user_id']=='6c991fe85d9fcaea917f71fbdc9e384e']

,user_id,age,sex,expect_quota,max_month_repay,occupation,education,marital_status,live_info,local_hk,...,business_year,personnel_num,pay_type,product_id,tm_encode,modifycount,tags,rels,cons,rel2count


In [216]:
ttt_0.describe()

,sex,expect_quota,max_month_repay,occupation,education,marital_status,live_info,local_hk,money_function,company_type,...,business_year,personnel_num,pay_type,product_id,tm_encode,modifycount,tags,rels,cons,rel2count
count,13141.000000,1.314100e+04,13141.000000,13141.000000,13141.000000,13141.000000,13141.000000,13141.000000,13141.000000,13141.000000,...,13141.000000,13141.000000,13141.000000,13141.000000,1.314100e+04,13141.000000,13141.000000,13141.000000,13141.000000,13141.000000
mean,1.155087,1.453981e+04,634.249220,2.507572,3.301423,1.906933,2.300510,1.756487,0.416254,2.530629,...,0.188875,-0.073206,0.392512,1.174416,1.679600e+07,6.470208,26.343505,236.065748,26.457804,52.408721
std,0.362212,3.193961e+04,5730.183975,0.884696,0.727324,0.994933,1.680995,1.639986,1.626809,2.331195,...,1.413331,0.580215,1.455725,0.379481,5.173721e+06,2.301174,28.310304,464.273595,39.622574,125.171426
min,0.000000,0.000000e+00,-1.000000,0.000000,0.000000,0.000000,0.000000,-1.000000,-1.000000,0.000000,...,-1.000000,-1.000000,-1.000000,1.000000,4.804428e+06,3.000000,0.000000,1.000000,0.000000,0.000000
25%,1.000000,5.000000e+03,-1.000000,2.000000,3.000000,1.000000,1.000000,1.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,1.000000,1.031519e+07,6.000000,0.000000,73.000000,3.000000,0.000000
50%,1.000000,1.000000e+04,-1.000000,2.000000,3.000000,1.000000,3.000000,2.000000,0.000000,3.000000,...,0.000000,0.000000,0.000000,1.000000,1.938969e+07,6.000000,23.000000,143.000000,13.000000,0.000000
75%,1.000000,2.000000e+04,-1.000000,3.000000,4.000000,3.000000,4.000000,3.000000,0.000000,5.000000,...,0.000000,0.000000,0.000000,1.000000,2.112359e+07,7.000000,41.000000,262.000000,34.000000,30.000000
max,2.000000,3.000000e+06,550000.000000,5.000000,4.000000,5.000000,6.000000,4.000000,10.000000,6.000000,...,7.000000,5.000000,5.000000,2.000000,2.185386e+07,83.000000,247.000000,18637.000000,771.000000,5636.000000


In [254]:
user_info['age'].unique()

array(['41', 'NONE', '46', '28', '34', '25', '33', '29', '31', '27', '39',
       '36', '37', '23', '35', '30', '26', '40', '45', '44', '32', '24',
       '22', '21', '42', '48', '20', '38', '19', '53', '47', '49', '43',
       '57', '51', '18', '17', '52', '50', '55', '54', '98', '64', '56',
       '66'], dtype=object)

In [283]:
train_user  = pd.merge(train,train_input_x,on='user_id',how='left')

In [284]:
array_feature = ['lable','sex','age','salary','education','marital_status','company_type','money_function','occupation','product_id','tags','rels','cons','modifycount']
train_user = train_user[array_feature]

In [285]:
train_user.corr()['lable']

lable             1.000000
sex               0.015212
salary            0.007925
education        -0.007624
marital_status    0.031464
company_type     -0.002416
money_function    0.019309
occupation        0.005891
product_id        0.008261
tags              0.003296
rels             -0.004895
cons             -0.010323
modifycount       0.026394
Name: lable, dtype: float64